In [ ]:
%pip install -q datasets huggingface_hub pandas librosa soundfile mutagen tqdm coqpit trainer

In [ ]:
# Bảo đảm import được gói TTS local trong repo
import sys, os

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.exists(os.path.join(repo_root, "TTS")) and repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Repo root:", repo_root)

In [ ]:
import os
import dotenv

dotenv.load_dotenv(os.path.join(repo_root, ".env"))
from huggingface_hub import login

token = os.environ.get("HF_TOKEN", "")
if not token:
    raise RuntimeError(
        "Set HF_TOKEN env var with a valid Hugging Face token that has access to capleaf/viVoice."
    )
login(token=token)


In [ ]:
from pathlib import Path
import os

DATASET_NAME = "capleaf/viVoice"
OUTPUT_ROOT = Path("data/viVoice").as_posix()
WAVS_DIR = os.path.join(OUTPUT_ROOT, "wavs")
META_FILE = "metadata.csv"
META_TRAIN_FILE = "meta_train.csv"
META_EVAL_FILE = "meta_eval.csv"
SAMPLE_RATE = 24000
MAX_SAMPLES = None  # đặt số nhỏ (vd 50000) nếu không muốn tải toàn bộ viVoice
EVAL_SPLIT = 0.02  # 2% làm validation

os.makedirs(WAVS_DIR, exist_ok=True)
print("Output root:", OUTPUT_ROOT)

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import librosa, soundfile as sf
import tqdm, os

# Cell 4: Tải dataset viVoice (streaming) và lưu về định dạng coqui: audio_file|text|speaker_name
# - Streaming để không tải toàn bộ về RAM cùng lúc (viVoice rất lớn)
# - Resample tất cả về 24kHz (SAMPLE_RATE) để đồng bộ với XTTS (XTTS dùng 24kHz)
# - Tách ngẫu nhiên thành train/eval theo EVAL_SPLIT (2% làm validation)
# - Ghi ra một file meta duy nhất (metadata.csv) theo định dạng coqui

ds = load_dataset(DATASET_NAME, split="train", streaming=True)
rows = []  # chỉ dùng một list, không tách train/eval ở đây
count = 0


def _get_text(ex):
    """Trích trường text từ một mẫu dataset (tên trường có thể khác nhau giữa các dataset)."""
    return (
        ex.get("text")
        or ex.get("sentence")
        or ex.get("transcript")
        or ex.get("transcription")
        or ex.get("normalized_text")
    )


def _get_speaker(ex):
    """Trích trường speaker_id/channel, nếu không có thì mặc định 'vivoice'."""
    return (
        ex.get("speaker")
        or ex.get("speaker_id")
        or ex.get("channel_id")
        or ex.get("channel")
        or "vivoice"
    )


for ex in tqdm.tqdm(ds, total=MAX_SAMPLES if MAX_SAMPLES else None):
    text = _get_text(ex)
    if not text:
        continue
    spk = _get_speaker(ex)

    # Lấy audio: có thể là dict với array hoặc path file
    audio = ex.get("audio") or ex.get("audio_raw") or ex.get("audio_data")
    wav_path = os.path.join(WAVS_DIR, f"{count:09d}.wav")
    if isinstance(audio, dict) and "array" in audio:
        y = np.asarray(audio["array"], dtype=np.float32)
        sr = audio.get("sampling_rate") or SAMPLE_RATE
    else:
        p = (
            ex.get("path")
            or ex.get("audio_filepath")
            or ex.get("audio_file")
            or ex.get("wav")
        )
        if not p:
            continue
        y, sr = librosa.load(p, sr=None, mono=True)
    # Resample về 24kHz nếu cần (XTTS dùng 24kHz)
    if sr != SAMPLE_RATE:
        y = librosa.resample(y, orig_sr=sr, target_sr=SAMPLE_RATE)
        sr = SAMPLE_RATE
    sf.write(wav_path, y, int(sr))

    rows.append(
        {
            "audio_file": f"wavs/{os.path.basename(wav_path)}",
            "text": str(text),
            "speaker_name": str(spk),
        }
    )
    count += 1
    if MAX_SAMPLES and count >= MAX_SAMPLES:
        break

# Ghi ra một file CSV theo định dạng coqui: audio_file|text|speaker_name
# Trainer sẽ tự tách train/eval nếu cần
meta_path = os.path.join(OUTPUT_ROOT, META_FILE)
pd.DataFrame(rows, columns=["audio_file", "text", "speaker_name"]).to_csv(
    meta_path, sep="|", index=False
)
meta_path

In [ ]:
from TTS.tts.configs.vits_config import VitsConfig
from TTS.config.shared_configs import BaseDatasetConfig

# Cell 5: Cấu hình VITS (giữ lại nếu bạn muốn thử VITS trước khi chuyển XTTS)
# - Nếu bạn muốn fine-tune XTTS, hãy bỏ qua cell này và chạy cell XTTS ở dưới
# - Nếu bạn muốn train VITS, giữ lại và chạy cell 6–7

cfg = VitsConfig()
cfg.audio.sample_rate = 24000
cfg.output_path = "train/vivoice_vits"
cfg.datasets = [
    BaseDatasetConfig(
        formatter="coqui",
        dataset_name="vivoice",
        path=OUTPUT_ROOT,
        meta_file_train=META_FILE,  # dùng metadata.csv đã tạo ở cell 4
        meta_file_val="",
        language="vi",
    )
]
cfg.use_phonemes = False
cfg.add_blank = True
cfg.model_args.use_speaker_embedding = True
cfg.num_loader_workers = 2
cfg.num_eval_loader_workers = 2
cfg

In [ ]:
from trainer import Trainer, TrainerArgs
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models import setup_model

# Cell 6: Train VITS (chỉ chạy nếu bạn muốn train VITS)
# - load_tts_samples sẽ tự động tách train/eval theo cfg.eval_split_size
# - setup_model khởi tạo VITS từ cfg
# - Trainer bắt đầu fit

train_samples, eval_samples = load_tts_samples(
    cfg.datasets,
    eval_split=True,
    eval_split_max_size=cfg.eval_split_max_size,
    eval_split_size=cfg.eval_split_size,
)
model = setup_model(cfg, train_samples + eval_samples)

train_args = TrainerArgs()
trainer = Trainer(
    train_args,
    model.config,
    cfg.output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    parse_command_line_args=False,
)
trainer.fit()

In [ ]:
from dataclasses import asdict
import json, os

# Cell 7: Lưu config VITS ra file JSON (để inference sau này)
# - Ghi cfg vào outputs/vivoice_vits/config.json
# - Có thể dùng để load model lại bằng VitsConfig.from_json(...)

os.makedirs(cfg.output_path, exist_ok=True)
with open(os.path.join(cfg.output_path, "config.json"), "w", encoding="utf-8") as f:
    json.dump(asdict(cfg), f, ensure_ascii=False, indent=2)
os.path.join(cfg.output_path, "config.json")

In [ ]:
# === PHẦN FINE-TUNE XTTS ===
# Nếu bạn muốn fine-tune XTTS thay vì VITS, hãy chạy các cell từ đây trở xuống

# Cell 8 (nếu cần): Kiểm tra lại đường dẫn file metadata.csv đã tạo ở cell 4
import os
print("Metadata CSV:", os.path.join(OUTPUT_ROOT, META_FILE))
print("Số mẫu:", len(pd.read_csv(os.path.join(OUTPUT_ROOT, META_FILE), sep="|")))

In [ ]:
from TTS.demos.xtts_ft_demo.utils.gpt_train import train_gpt

# Cell 9: Cấu hình fine-tune XTTS trên viVoice
# - LANGUAGE: ngôn ngữ của dataset (vi cho tiếng Việt)
# - NUM_EPOCHS: số epoch train, tăng lên 20–50 cho train dài
# - BATCH_SIZE: batch size cho mỗi step; 4 an toàn hơn cho GPU 12–16GB
# - GRAD_ACUMM: gradient accumulation, giúp effective batch = BATCH_SIZE * GRAD_ACUMM
# - MAX_AUDIO_SECONDS: độ dài audio tối đa mỗi mẫu (giữ 20s để dùng nhiều dữ liệu dài hơn)
# - TRAIN_CSV_PATH / EVAL_CSV_PATH: cùng trỏ vào metadata.csv (train_gpt sẽ tự tách)
# - OUTPUT_XTTS_ROOT: thư mục chứa checkpoint và log của XTTS fine-tune

LANGUAGE = "vi"
NUM_EPOCHS = 10
BATCH_SIZE = 4
GRAD_ACUMM = 2
MAX_AUDIO_SECONDS = 20

TRAIN_CSV_PATH = os.path.join(OUTPUT_ROOT, META_FILE)  # dùng chung metadata.csv
EVAL_CSV_PATH = os.path.join(OUTPUT_ROOT, META_FILE)    # train_gpt sẽ tự tách
OUTPUT_XTTS_ROOT = "outputs/vivoice_xtts"

print("Train CSV:", TRAIN_CSV_PATH)
print("Eval  CSV:", EVAL_CSV_PATH)
print("Output :", OUTPUT_XTTS_ROOT)

In [ ]:
# Cell 10: Chạy fine-tune XTTS GPT trên viVoice
# - train_gpt() sẽ tự động:
#   1. Tải các pre‑trained model XTTS (dvae, mel_stats, vocab, model.pth, config.json)
#   2. Tạo GPTTrainerConfig với batch_size, grad_accum, num_loader_workers, ...
#   3. Tạo dataset từ CSV theo formatter "coqui"
#   4. Khởi tạo GPTTrainer và bắt đầu fine‑tune GPT của XTTS
# - Trả về:
#   - config_path: config XTTS gốc (để inference)
#   - xtts_checkpoint: model XTTS gốc (để inference)
#   - vocab_file: vocab tokenizer
#   - exp_path: thư mục run chứa checkpoint fine‑tuned (best_model.pth)
#   - speaker_ref: file wav dài nhất dùng làm reference khi inference

config_path, xtts_checkpoint, vocab_file, exp_path, speaker_ref = train_gpt(
    LANGUAGE,
    NUM_EPOCHS,
    BATCH_SIZE,
    GRAD_ACUMM,
    TRAIN_CSV_PATH,
    EVAL_CSV_PATH,
    output_path=OUTPUT_XTTS_ROOT,
    max_audio_length=int(MAX_AUDIO_SECONDS * 22050),
)

print("XTTS config :", config_path)
print("XTTS ckpt   :", xtts_checkpoint)
print("Vocab file  :", vocab_file)
print("Run folder  :", exp_path)
print("Speaker ref :", speaker_ref)

config_path, xtts_checkpoint, vocab_file, exp_path, speaker_ref

In [ ]:
# Cell 11 (tùy chọn): Inference thử với XTTS đã fine‑tune
# - Dùng config_path, xtts_checkpoint, vocab_file, exp_path từ cell 10
# - speaker_ref là file wav dùng làm giọng tham chiếu
# - Ví dụ:
#   from TTS.tts.configs.xtts_config import XttsConfig
#   from TTS.tts.models.xtts import Xtts
#   import soundfile as sf
#
#   cfg = XttsConfig()
#   cfg.load_json(config_path)
#   model = Xtts.init_from_config(cfg)
#   model.load_checkpoint(cfg, checkpoint_path=os.path.join(exp_path, "best_model.pth"), vocab_path=vocab_file)
#   model.cuda()
#
#   text = "Xin chào, đây là thử nghiệm giọng nói tiếng Việt."
#   wav = model.full_inference(text, speaker_ref, language="vi")["wav"]
#   sf.write("out.wav", wav, 24000)
#
# # Bỏ comment để chạy khi đã có checkpoint